# Demo for SPEL using LakeTemperature


In [2]:
%tb
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Built-in modules
import re # Regular expressions
import importlib # MUST BE USED TO RELOAD MODULES

# Importing modules for SPEL functions 
import utilityFunctions as uf
import edit_files as ef 
import fortran_modules as fm 
from analyze_subroutines import Subroutine
# mod_config : system configuration and static variables. 
# Where E3SM is stored, where unit tests are stored, etc.
from mod_config import default_mods, unittests_dir, scripts_dir, spel_mods_dir
from mod_config import ELM_SRC, spel_output_dir, _bc


NameError: name 'unittests_dir' is not defined

SPEL needs a "casename" (what your unit-test is called) and a list of subroutines to create a unit-test for "sub_name_list".  For this demo, the sub_name_list only has "LakeTemperature" in it.


In [3]:
# Define Unit Test parameters 
casename = "LakeTemp"  # Name of the test case
case_dir = unittests_dir + casename # Directory to store the test case

# List of subroutines to be analyzed
sub_name_list = ["LakeTemperature"]
sub_name_list = [sub.lower() for sub in sub_name_list]

# variables needed for Unit Test
main_sub_dict = {}  # Dictionary to store all Subroutines in files needed for Unit Test
subroutines = {k.lower():[] for k in sub_name_list} # Dictionary for User Specified Subroutines

1st step for SPEL is determine which modules are needed for LakeTemperature and edit out the I/O, MPI, and other unneccessary modules. This must be somewhat tailored to ELM.

Currently if I module is not present in the "components/elm/src/" (ELM_SRC) nor in the "share/utils" directories (SHR_SRC), then the module and any dependency on it is automatically removed.

In [4]:
 # List to hold all the modules needed for the unit test
needed_mods = [] 
for s in sub_name_list:
    # Get general info of the subroutine
    subroutines[s] = Subroutine(s,calltree=['elm_drv'])

    # Process by removing certain modules and syntax
    # so that a standalone unit test can be compiled.
    fn = subroutines[s].filepath
    mod_dict, file_list = ef.process_for_unit_test(fname=fn,case_dir=case_dir,
                            mods=needed_mods,required_mods=default_mods, 
                            main_sub_dict=main_sub_dict,
                            overwrite=False,verbose=False)
    subroutines[s] = main_sub_dict[s]

Couldn't find shr_megan_mod in ELM or shared source -- adding to removal list
Couldn't find shr_fan_mod in ELM or shared source -- adding to removal list
Couldn't find petscsys in ELM or shared source -- adding to removal list
Couldn't find petscvec in ELM or shared source -- adding to removal list
Couldn't find petscmat in ELM or shared source -- adding to removal list
Couldn't find petscts in ELM or shared source -- adding to removal list
Couldn't find petscdm in ELM or shared source -- adding to removal list
Couldn't find petscdmda in ELM or shared source -- adding to removal list
Couldn't find unstructuredgridtype in ELM or shared source -- adding to removal list
Couldn't find mathfuncmod in ELM or shared source -- adding to removal list
Couldn't find tracer_varcon in ELM or shared source -- adding to removal list
Couldn't find fatesinterfacetypesmod in ELM or shared source -- adding to removal list


In [5]:
# Create dictionary containing all identified user defined types
# 'type-name' : 'DerivedType Object'
type_dict = {}
for modname, mod in mod_dict.items():
    for utype, dtype in mod.defined_types.items():
        type_dict[utype] = dtype

SPEL can print out a module tree showing how the modules are linked for the unit-test subroutines
- The full tree can be pretty difficult to parse, having a cutoff depth is recommended.
- Every module uses 'shr_kind_mod' so that could be suppressed as well.

In [ ]:
importlib.reload(fm)
modtree = fm.print_spel_module_dependencies(mod_dict=mod_dict,subs=subroutines)

arrow = "-->"
cutoff_depth = 10 # Only print modules up to this depth
suppress_mod_list = ['shr_kind_mod']
for m in modtree:
    depth = m['depth']
    modname = m['module']
    if(modname in suppress_mod_list):
        continue
    if(depth == 1):
        print(_bc.HEADER + arrow*depth + modname + _bc.ENDC)
    elif(depth <= cutoff_depth):
        print( arrow*depth + modname)

In [ ]:
test_mod = mod_dict['laketemperaturemod']
test_mod.display_info()

In [ ]:
for f in file_list:
    base_fn = f.split('/')[-1]
    print(base_fn)

In [ ]:
# for sub in main_sub_dict.values():
#     print(sub.name)
sub = main_sub_dict['laketemperature']
sub.printSubroutineInfo()

In [ ]:
sub.LocalVariables['arrays']

In [6]:
for s in sub_name_list:
    # Parsing means getting info on the variables read and written
    # to by the subroutine and any of its callees
    subroutines[s].parse_subroutine(dtype_dict=type_dict,
                                    main_sub_dict=main_sub_dict,verbose=True)

Resolving interface for tridiagonal
 with args: ['bounds', '-nlevsno + 1', 'nlevlak + nlevgrnd', 'jtop', 'num_lakec', 'filter_lakec', 'a', 'b', 'c1', 'r', 'tx']
/home/mrgex/SPEL_Openacc/scripts/../../repo/E3SM/components/elm/src/biogeophys/TridiagonalMod.F90 16   interface Tridiagonal
Couldn't match -nlevsno + 1 to any known variable -- assuming xx type
Couldn't match nlevlak + nlevgrnd to any known variable -- assuming xx type
tridiagonal_sr:: 1 Optional arguments found
bounds matches bounds
-nlevsno + 1 matches lbj
nlevlak + nlevgrnd matches ubj
jtop matches jtop
num_lakec matches numf
filter_lakec matches filter
a matches a
b matches b
c1 matches c
r matches r
tx matches u
resolve_interface::Subroutine is tridiagonal_sr
_preprocess_file::New child sub name is: tridiagonal_sr
_preprocess_file::Finished analyzing for laketemperature


In [9]:
read_types  = [] 
write_types = []

for s in sub_name_list:
    subroutines[s].child_subroutines_analysis(dtype_dict=type_dict,
                                        main_sub_dict=main_sub_dict,verbose=True)
    print(_bc.OKGREEN+f"Derived Type Analysis for {subroutines[s].name}")
    print(f"Read-Only")
    for key in subroutines[s].elmtype_r.keys():
        print(key, subroutines[s].elmtype_r[key])
    print(f"Write-Only")
    for key in subroutines[s].elmtype_w.keys():
        print(key, subroutines[s].elmtype_w[key])
    print(f"Read-Write")
    for key in subroutines[s].elmtype_rw.keys():
        print(key, subroutines[s].elmtype_rw[key])
    print(_bc.ENDC)
    for key in subroutines[s].elmtype_r.keys():
        c13c14 = bool('c13' in key or 'c14' in key)
        if(c13c14):
            del subroutines[s].elmtype_r[key]
            continue
        if("_inst" in key):
           print(f"error: {key} has _inst")
           sys.exit(1)
        read_types.append(key)
        
    for key in subroutines[s].elmtype_w.keys():
        c13c14 = bool('c13' in key or 'c14' in key)
        if(c13c14):
            del subroutines[s].elmtype_w[key]
            continue
        if("_inst" in key):
            print(f"error: {key} has _inst")
            sys.exit(1)
        write_types.append(key)
        
    for key in subroutines[s].elmtype_rw.keys():
        c13c14 = bool('c13' in key or 'c14' in key)
        if(c13c14):
            del subroutines[s].elmtype_w[key]
            continue
        write_types.append(key)

Analyzing child subroutine: soilthermprop_lake
_preprocess_file::Finished analyzing for soilthermprop_lake
Analyzing child subroutine: tridiagonal_sr
_preprocess_file::Finished analyzing for tridiagonal_sr
Analyzing child subroutine: phasechange_lake
_preprocess_file::Finished analyzing for phasechange_lake
Derived Type Analysis for laketemperature
Read-Only
bounds%begc r
bounds%endc r
col_pp%snl r
veg_pp%column r
solarabs_vars%fsds_nir_d_patch r
solarabs_vars%fsds_nir_i_patch r
solarabs_vars%fsr_nir_d_patch r
solarabs_vars%fsr_nir_i_patch r
solarabs_vars%sabg_patch r
col_pp%z_lake r
lakestate_vars%ws_col r
lakestate_vars%ks_col r
col_pp%z r
col_es%t_grnd r
col_pp%lakedepth r
lakestate_vars%etal_col r
col_pp%dz_lake r
col_pp%dz r
lakestate_vars%lake_raw_col r
soilstate_vars%tksatu_col r
soilstate_vars%tkmg_col r
soilstate_vars%watsat_col r
soilstate_vars%tkdry_col r
col_pp%zi r
soilstate_vars%csol_col r
Write-Only
col_ws%frac_iceold w
lakestate_vars%savedtke1_col w
col_ef%eflx_snomelt 

In [10]:
# Make sure physical properties types are read/written:
list_pp = ['veg_pp','lun_pp','col_pp','grc_pp','top_pp']

print("read_types:",read_types)
print("write_types:",write_types)

aggregated_elmtypes_list = []
for x in read_types:
    dtype_inst = x.split('%')[0]
    if(dtype_inst not in aggregated_elmtypes_list):
        aggregated_elmtypes_list.append(dtype_inst)    
for x in write_types:
    dtype_inst = x.split('%')[0]
    if(dtype_inst not in aggregated_elmtypes_list):
        aggregated_elmtypes_list.append(dtype_inst)

# for l in list_pp:
#     aggregated_elmtypes_list.append(l)
print("list of global vars:",aggregated_elmtypes_list)

read_types: ['bounds%begc', 'bounds%endc', 'col_pp%snl', 'veg_pp%column', 'solarabs_vars%fsds_nir_d_patch', 'solarabs_vars%fsds_nir_i_patch', 'solarabs_vars%fsr_nir_d_patch', 'solarabs_vars%fsr_nir_i_patch', 'solarabs_vars%sabg_patch', 'col_pp%z_lake', 'lakestate_vars%ws_col', 'lakestate_vars%ks_col', 'col_pp%z', 'col_es%t_grnd', 'col_pp%lakedepth', 'lakestate_vars%etal_col', 'col_pp%dz_lake', 'col_pp%dz', 'lakestate_vars%lake_raw_col', 'soilstate_vars%tksatu_col', 'soilstate_vars%tkmg_col', 'soilstate_vars%watsat_col', 'soilstate_vars%tkdry_col', 'col_pp%zi', 'soilstate_vars%csol_col']
write_types: ['col_ws%frac_iceold', 'lakestate_vars%savedtke1_col', 'col_ef%eflx_snomelt', 'col_ef%imelt', 'col_es%hc_soi', 'lakestate_vars%lakeresist_col', 'veg_ef%eflx_gnet', 'lakestate_vars%betaprime_col', 'lakestate_vars%lake_icefrac_col', 'col_es%t_lake', 'col_es%t_soisno', 'ch4_vars%grnd_ch4_cond_col', 'col_ef%errsoi', 'veg_ef%eflx_sh_tot', 'veg_ef%eflx_sh_grnd', 'veg_ef%eflx_soil_grnd', 'lakestat

In [11]:
from UnitTestforELM import set_active_variables
instance_to_user_type = {}
elm_inst_vars = {}
for type_name, dtype in type_dict.items():
    if('bounds' in type_name): 
        continue
    if(not dtype.instances):
        print(f"Warning: no instances found for {type_name}")
        cmd = f'grep -rin -E "^[[:space:]]*(type)[[:space:]]*\({type_name}" {ELM_SRC}/main/elm_instMod.F90'
        output = sp.getoutput(cmd)
        print(f"output: {output}")
        if(output):
            output = output.split('\n')
            if(len(output) > 1):
                print(f"Warning: multiple instances found for {type_name}")
                print(output)
                sys.exit(1)
            line = output[0]
            line = line.replace('::','')
            line = line.split(':')
            
            decl = line[1].strip()
            decl = decl.split()
            var = decl[1]
            new_inst = Variable(type_name,var,subgrid='?',ln=0,dim=0,declaration='elm_instMod')
            dtype.instances.append(new_inst)
            elm_inst_vars[var] = dtype
        else:
            print(f"Warning: no instances found for {type_name}")
    for instance in dtype.instances:
        instance_to_user_type[instance.name] = type_name

dtype_info_list = []
    
for s in sub_name_list:
    set_active_variables(type_dict,instance_to_user_type,
                            subroutines[s].elmtype_r,dtype_info_list)
    set_active_variables(type_dict,instance_to_user_type,
                            subroutines[s].elmtype_w,dtype_info_list)
    set_active_variables(type_dict,instance_to_user_type,
                            subroutines[s].elmtype_rw,dtype_info_list)
    
print(dtype_info_list)

[['col_pp', 'snl', 'integer', '1D'], ['veg_pp', 'column', 'integer', '1D'], ['solarabs_vars', 'fsds_nir_d_patch', 'real', '1D'], ['solarabs_vars', 'fsds_nir_i_patch', 'real', '1D'], ['solarabs_vars', 'fsr_nir_d_patch', 'real', '1D'], ['solarabs_vars', 'fsr_nir_i_patch', 'real', '1D'], ['solarabs_vars', 'sabg_patch', 'real', '1D'], ['col_pp', 'z_lake', 'real', '2D'], ['lakestate_vars', 'ws_col', 'real', '1D'], ['lakestate_vars', 'ks_col', 'real', '1D'], ['col_pp', 'z', 'real', '2D'], ['col_es', 't_grnd', 'real', '1D'], ['col_pp', 'lakedepth', 'real', '1D'], ['lakestate_vars', 'etal_col', 'real', '1D'], ['col_pp', 'dz_lake', 'real', '2D'], ['col_pp', 'dz', 'real', '2D'], ['lakestate_vars', 'lake_raw_col', 'real', '1D'], ['soilstate_vars', 'tksatu_col', 'real', '2D'], ['soilstate_vars', 'tkmg_col', 'real', '2D'], ['soilstate_vars', 'watsat_col', 'real', '2D'], ['soilstate_vars', 'tkdry_col', 'real', '2D'], ['col_pp', 'zi', 'real', '2D'], ['soilstate_vars', 'csol_col', 'real', '2D'], ['col